In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config=ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=0.5
config.gpu_options.allow_growth=True
session=InteractiveSession(config=config)

In [ ]:
import pandas as pd
from tensorflow.keras.optimizers import RMSprop,Adam
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras import Model
from os import getcwd
from keras.layers import Dense, Activation, BatchNormalization
import tensorflow as tf

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='val_accuracy', patience = 15, verbose=0, mode='max')
mcp_save_xc = ModelCheckpoint(filepath='mdl_wts.hdf5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens = ModelCheckpoint(filepath='mdl_wts_en.h5', save_best_only=True, monitor='val_accuracy', mode='max')
mcp_save_ens1 = ModelCheckpoint(filepath='mdl_wts_en1.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
train_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")

In [ ]:
train_df.head()

In [ ]:
train_df['label'].value_counts()

In [ ]:
train_df['label']=train_df['label'].astype(str)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = "../input/cassava-leaf-disease-classification/train_images"
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    validation_split=0.05,
    #zoom_range=.2,

    horizontal_flip=True,
    #fill_mode='nearest'
)
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAINING_DIR,
    x_col="image_id",
    y_col="label",
    batch_size = 64,
    subset = 'training',
    class_mode='categorical',
    shuffle = True,
    target_size=(224,224)
)


validation_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
    directory = TRAINING_DIR,
    x_col="image_id",
    y_col="label",
    shuffle = True,
    batch_size = 64,
    subset = 'validation',
    class_mode='categorical',
    target_size=(224,224)
)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
pre_model = InceptionV3( input_shape = (224,224,3), include_top = False )

In [ ]:
for layer in pre_model.layers[:20]:
  layer.trainable = False

In [ ]:
pre_model.summary()

In [ ]:
last_layer = pre_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras.optimizers import RMSprop,Adam

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
from keras.layers import Dense, Activation

x = layers.Flatten()(last_output)

x = layers.Dropout(0.5)(x)

x = layers.Dense(1024, activation = "relu")(x)

x = layers.Dense(1024, activation = "relu")(x)

x = layers.Dropout(0.5)(x)

x = layers.Dense(5, activation = "softmax")(x)

model_new = Model(pre_model.input, x)

model_new.compile(optimizer = Adam(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy']
             )

In [ ]:
history_xc = model_new.fit(
      train_generator,  
      callbacks = [ mcp_save_xc, reduce_lr_loss],
      validation_data = validation_generator,
      epochs = 15,
      verbose = 1,
 )

In [ ]:
\